In [0]:
#Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [0]:
results_df = spark.read.json("dbfs:/FileStore/results.json" , schema = results_schema )

In [0]:
#Step 2 - Rename columns and add new columns

In [0]:
from pyspark.sql.functions import lit , current_timestamp

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") 
                                    

In [0]:
results_with_ingestion_date_df = results_with_columns_df.withColumn("ingestion_date", current_timestamp())

In [0]:
#Step 3 - Drop the unwanted column

In [0]:
from pyspark.sql.functions import col

In [0]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

In [0]:
results_deduped_df = results_final_df.dropDuplicates(['race_id', 'driver_id'])

In [0]:
results_deduped_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+---------+------------+-----------+----+----------------+-----------------+----------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time     |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|ingestion_date        |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+---------+------------+-----------+----+----------------+-----------------+----------------------+
|7573     |1      |1        |1             |1     |18  |null    |D            |20            |0.0   |58  |\N       |null        |39         |13  |1:29.020        |214.455          |2024-04-23 04:09:58.81|
|7563     |1      |2        |2             |6     |9   |10      |10           |10            |0.0   |58  |+7.085   |5662869     |48         |5   |1:28.283        |216.245          

In [0]:
#Step 4 - Write to output to processed container in parquet format

In [0]:
results_deduped_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.result") 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2747915330769759>:1
----> 1 results_deduped_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.result")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321,